## Installation

Installing Bio, transformers, genomic-benchmarks, and datasets packages.  The Bio package is from Biopython; transformers package for machine learning (pytorch, tensorflow); genomic-benchmarks and datasets from ML-Bioinfo-CEITEC.


In [3]:
!pip install -qq Bio transformers genomic-benchmarks datasets transformers[torch] evaluate scikit-learn optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 78.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 104.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 27.5 MB/s eta 0:00:00


Access TFBS dataset in Google Drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Set kmer and model parameters

In [5]:
kmer = 6
model_used = "armheb/DNA_bert_" + str(kmer)
bs = 64
epochs = 4
warmup = 100
maxstp = 1000
lr = 8e-5

Testing parameter changes on 3 datasets

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from datasets import Dataset, DatasetDict, load_metric
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate
import optuna
import sys
import os

# set output path

stdpath = "/content/drive/MyDrive/DNABERT/output/" + "optimization" + "/"

# pick a few TFBS datasets with varying accuracy

fname = "HaibA549Yy1cV0422111Etoh02UniPk-ran.csv"

dsname = "/content/drive/MyDrive/DNABERT/Datasets/tfbs/" + fname
tfbs_dataset = pd.read_csv(dsname, sep=',')

# change standard error and output to saved output file

try:
  os.makedirs(stdpath)
except FileExistsError:
  pass

stdpath_out = stdpath + "stdout.txt"
stdpath_err = stdpath + "stderr.txt"

sys.stdout = open(stdpath_out, 'w')
sys.stderr = open(stdpath_err, 'w')

# reformat input

column_names = ["labels", "seq"]
tfbs_dataset_res = pd.DataFrame(columns=column_names)

j = 1
for i in range(tfbs_dataset.count()['names']):
  name = tfbs_dataset['names'][i]
  pos_seq = tfbs_dataset['seq'][i]
  neg_seq = tfbs_dataset['neg_seq'][i]
  pos_label = 1
  neg_label = 0
  tfbs_dataset_res.loc[j] = [pos_label, pos_seq]
  tfbs_dataset_res.loc[j+1] = [neg_label, neg_seq]
  j+=2

X = tfbs_dataset_res['seq']
y = tfbs_dataset_res['labels']
X_size = len(X)

# test range is 500 odd-numbered pairs of sequences
# training range is 500 even-numbered pairs plus remainder of the sequences over 1000

test_range = list(range(3,1001,4)) + list(range(4,1001,4))
test_range.sort()
train_range = list(range(1,1001,4)) + list(range(2,1001,4))
train_range.sort()
train_range = train_range + list(range(1001,X_size))

X_test = X.loc[test_range]
X_train = X.loc[train_range]
y_test = y.loc[test_range]
y_train = y.loc[train_range]

# print size of training and test sets

print("X_train shape:", X_train.shape, file=sys.stdout)
print("y_train shape:", y_train.shape, file=sys.stdout)
print("X_test shape:", X_test.shape, file=sys.stdout)
print("y_test shape:", y_test.shape, file=sys.stdout)

# tokenization

def kmers_stride1(s, k=kmer):
  return [s[i:i + k] for i in range(0, len(s)-k+1)]




Some weights of BertForSequenceClassification were not initialized from the model checkpoint at armheb/DNA_bert_4 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/20017 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.599900,0.540353,0.752000,0.677083
2,0.433500,0.413950,0.802000,0.811429
3,0.355500,0.484349,0.808000,0.825455
4,0.229800,0.481835,0.824000,0.834586


...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at armheb/DNA_bert_4 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/45883 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.193800,0.176480,0.942000,0.943470
2,0.108600,0.186381,0.956000,0.957031
3,0.070000,0.167488,0.960000,0.960784
4,0.055400,0.182025,0.958000,0.958904


In [ ]:
# set optuna hyperparameter trial space

def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16, 32, 64, 128]),
        "warmup_steps": trial.suggest_int("warmup_steps", 50, 150)
    }

In [ ]:
# define model initialization

def model_init(trial):
    return AutoModelForSequenceClassification.from_pretrained(model_used, num_labels=2)

In [ ]:
# load pre-trained model

model_cls = AutoModelForSequenceClassification.from_pretrained(model_used, num_labels=2)
params = list(model_cls.named_parameters())
tokenizer = AutoTokenizer.from_pretrained(model_used)

# reformat data to Hugging Face Dataset format from pandas

ds_Xy_train = pd.concat([y_train, X_train], axis=1)
ds_Xy_test = pd.concat([y_test, X_test], axis=1)

Dataset_Xy_train = Dataset.from_pandas(ds_Xy_train)
Dataset_Xy_test = Dataset.from_pandas(ds_Xy_test)
Dataset_Xy_train, Dataset_Xy_test

def tok_func(x): return tokenizer(" ".join(kmers_stride1(x["seq"])))

Dataset_Xy_train_tok = Dataset_Xy_train.map(tok_func, batched=False)
new_column = ["train"] * len(Dataset_Xy_train_tok)
Dataset_Xy_train_tok = Dataset_Xy_train_tok.add_column("dset", new_column)

Dataset_Xy_test_tok = Dataset_Xy_test.map(tok_func, batched=False)
new_column = ["test"] * len(Dataset_Xy_test_tok)
Dataset_Xy_test_tok = Dataset_Xy_test_tok.add_column("dset", new_column)

dds = DatasetDict({
  'train': Dataset_Xy_train_tok,
  'test': Dataset_Xy_test_tok
})

# switch to GPU

if torch.cuda.device_count() > 0:
  model_cls.to('cuda')

# train model

args = TrainingArguments('outputs', learning_rate=lr, warmup_steps=warmup, max_steps=maxstp, lr_scheduler_type='cosine', fp16=True,
                         evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
                         num_train_epochs=epochs, weight_decay=0.01, report_to='none')

def compute_metrics(eval_preds):
  metric = evaluate.combine(["accuracy","precision","recall","f1","matthews_correlation"])
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(model=None, args=args, train_dataset=dds['train'], eval_dataset=dds['test'],tokenizer=tokenizer, compute_metrics=compute_metrics, model_init=model_init)


In [ ]:
best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=optuna_hp_space,
    n_trials=20
)

In [ ]:
best_trial

In [ ]:
best_trial.hyperparameters